In this post we'll look at a fourth-order equation describing the vibrations of clamped plates.
Our starting point are the equations of linear elasticity.
Here we want to solve for the 3D displace $u$ of the medium.
The first equation is momentum conservation:
$$\rho\ddot u = \nabla\cdot\sigma + f$$
where $\sigma$ is the stress tensor and $f$ the body forces.
To close the system, we need to supply a constitutive equation relating the stress tensor and the strain tensor
$$\varepsilon = \frac{1}{2}\left(\nabla u + \nabla u^*\right).$$
The most general linear constitutive equation we can write down is
$$\sigma = \mathscr C\,\varepsilon$$
where $\mathscr C$ is the rank-4 elasticity tensor.
We need that $\mathscr C$ maps symmetric tensors to symmetric tensors and that $\varepsilon:\mathscr C\varepsilon$ is always positive.
The equilibrium displacement of the medium is the minimizer of the energy functional
$$J(u) = \int_\Omega\left(\frac{1}{2}\sigma:\varepsilon + f\cdot u\right)\mathrm dx + \ldots$$
where the ellipses stand for various boundary forcings that we'll ignore.

For a medium that is homogeneous and isotropic, the elasticity tensor has to have the form
$$\mathscr C\,\varepsilon = 2\,\mu\,\varepsilon + \lambda\,\text{tr}(\varepsilon)I$$
where $\mu$ and $\lambda$ are the Lamé parameters.
As an aside, there are a mess of alternate forms of the elasticity equations.
The [wiki page](https://en.wikipedia.org/wiki/Lam%C3%A9_parameters) has a conversion table at the bottom.
Now take this with a grain of salt because I do fluid mechanics.
But if I were a solid mechanics kinda guy, this would embarrass me.
Get it together folks.


Plate theory is what you get when you assume the medium is thin along the vertical dimension and that this restricts the form that the displacements can take.
The first and most widely agreed-upon simplification is that the vertical displacement is some function $w$ of the horizontal coordinates $x$, $y$:
$$u_z = w(x, y).$$
From here we have to make additional assumptions about the horizontal displacements $u_x$ and $u_y$.
Different theories make different sets of assumptions.
The classical theory is the [Kirchoff-Love](https://en.wikipedia.org/wiki/Kirchhoff%E2%80%93Love_plate_theory) plate.
The Kirchoff theory assumes that any straight line that's perpendicular to the middle of the plate remains straight and perpendicular after deformation.
These assumptions let us write down the other components of the deformation in terms of $w$:
$$u_x = -z\frac{\partial w}{\partial x}, \quad u_y = -z\frac{\partial w}{\partial y}.$$
It's possible that $u_x$ and $u_y$ are also offset by in-plane displacements.
I'll assume that the boundary conditions make those equal to zero.
We can then work out what the displacement gradient is:
$$\nabla u = \left[\begin{matrix}-z\nabla^2 w & -\nabla w \\ +\nabla w^* & 0 \end{matrix}\right]$$
I'm being a little loose in my use of the gradient operator -- it's 3D on the left-hand side of this equation and 2D on the right.
Notice how the antisymmetric part of the displacement gradient is all in the $x-z$ and $y-z$ components.
When we symmetrize the displacement gradient, we get the strain tensor:
$$\varepsilon = -z\left[\begin{matrix}\nabla^2 w & 0 \\ 0 & 0\end{matrix}\right].$$
Now remember that the medium is a thin plate.
We can express the spatial domain as the product of a 2D footprint domain $\Phi$ and the interval $[-h / 2, +h / 2]$ where $h$ is the plate thickness.
The strain energy is then
$$\begin{align}
J(w) & = \int_\Phi\int_{-h/2}^{+h/2}\left(\mu z^2|\nabla^2 w|^2 + \frac{\lambda}{2} z^2|\Delta w|^2 + fw\right)\mathrm dz\;\mathrm dx\\
 & = \int_\Phi\left\{\frac{h^3}{24}\left(2\mu|\nabla ^2 w|^2 + \lambda|\Delta w|^2\right) + hfw\right\}\mathrm dx
\end{align}.$$
Here I've used the fact that $\text{tr}(\nabla^2w) = \Delta w$ where $\Delta$ is the Laplace operator.
**Note to self check all the constants and factors of 2 and such.
Also check out the references in [this paper](https://www-users.cse.umn.edu/~arnold//papers/rmkl.pdf) about this extra Laplacian term, WTF.**


We can't discretize this problem right away using a conventional finite element basis.
The usual piecewise polynomial basis functions are differentiable and piecewise continuous across cell edges.
A conforming basis for a minimization problem involving 2nd-order derivatives would need to instead by continuously differentiable.
It's much harder to come up with $C^1$ bases.

We have a few ways out.
1. Use a $C^1$ basis like [Argyris](https://defelement.org/elements/argyris.html) or [Hsieh-Clough-Tocher](https://defelement.org/elements/hsieh-clough-tocher.html).
This approach makes forming the problem easy, but applying boundary conditions hard.
[Kirby and Mitchell (2018)](https://dl.acm.org/doi/pdf/10.1145/3361745) implemented the Argyris element in Firedrake but had to use Nitsche's method to enforce the boundary conditions.
1. Use $C^0$ elements and discretize the second derivatives using an interior penalty formulation of the problem.
This approach is analogous to discretizing a 2nd-order elliptic problem using DG elements.
Forming the problem is harder but applying the boundary conditions is easier.
[Bringmann et al. (2023)](https://doi.org/10.1515/jnma-2023-0028) work out the exact form the interior penalty parameters necessary to make the discrete form work right.
1. Use the mixed form of the problem and discretize it with the [Hellan-Herrmann-Johnson](https://defelement.org/elements/hellan-herrmann-johnson.html) element.

In the following, I'll take the third approach.

In [ ]:
import firedrake
from firedrake import Constant, as_vector, sym, inner, grad, div, dx, ds

nx, ny = 32, 32
mesh = firedrake.UnitSquareMesh(nx, ny, diagonal="crossed")

### Eigenfunctions of the clamped plate

Also known as Chladni patterns.